In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "google-bert/bert-base-uncased"

In [3]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_CB")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model.config.use_cache = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"

def preprocess_function(examples):
    """
    The preprocessing function prepares examples for processing by the model.
    It concatenates premise and hypothesis for each example to form a single input string.
    """
    inputs = [f"{premise} {hypothesis}" for premise, hypothesis in zip(examples[CONTEXT_COL], examples[HYPOTHESIS_COL])]
    tokenized_examples = tokenizer(inputs, truncation=True)
    if "label" in examples:
        tokenized_examples["labels"] = [label2id[label] for label in examples["label"]]
    return tokenized_examples

In [7]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'hypothesis', 'label'], batched=True)

In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
})

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
import numpy as np

from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [11]:
from transformers import set_seed

set_seed(42)

In [12]:
from peft import LoHaConfig, get_peft_model, TaskType

loha_alpha = 32
loha_rank_dropout = 0.1
loha_module_dropout = 0.0
loha_r = 16

peft_config = LoHaConfig(
    task_type=TaskType.SEQ_CLS,
    r=loha_r,
    alpha=loha_alpha,
    target_modules=['query', 'value'],
    base_model_name_or_path=model_name
)

In [13]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,181,955 || all params: 110,666,502 || trainable%: 1.0680


In [14]:
from transformers import TrainingArguments

new_model_name = "loha_fine_tuned_cb"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [16]:
trainer.train()

  0%|          | 0/400 [00:00<?, ?it/s]

{'loss': 1.0888, 'grad_norm': 9.729341506958008, 'learning_rate': 1.7500000000000002e-05, 'epoch': 3.57}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.0481975078582764, 'eval_accuracy': 0.2727272727272727, 'eval_f1': 0.13636363636363635, 'eval_runtime': 0.8336, 'eval_samples_per_second': 26.391, 'eval_steps_per_second': 3.599, 'epoch': 3.57}
{'loss': 0.8883, 'grad_norm': 6.1059746742248535, 'learning_rate': 1.5000000000000002e-05, 'epoch': 7.14}


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.3167388439178467, 'eval_accuracy': 0.3181818181818182, 'eval_f1': 0.1536050156739812, 'eval_runtime': 0.6273, 'eval_samples_per_second': 35.072, 'eval_steps_per_second': 4.783, 'epoch': 25.0}
{'loss': 0.7395, 'grad_norm': 4.701554775238037, 'learning_rate': 0.0, 'epoch': 28.57}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.3193508386611938, 'eval_accuracy': 0.3181818181818182, 'eval_f1': 0.1536050156739812, 'eval_runtime': 0.341, 'eval_samples_per_second': 64.511, 'eval_steps_per_second': 8.797, 'epoch': 28.57}
{'train_runtime': 174.1955, 'train_samples_per_second': 18.37, 'train_steps_per_second': 2.296, 'train_loss': 0.8083693790435791, 'epoch': 28.57}


TrainOutput(global_step=400, training_loss=0.8083693790435791, metrics={'train_runtime': 174.1955, 'train_samples_per_second': 18.37, 'train_steps_per_second': 2.296, 'total_flos': 307420818029424.0, 'train_loss': 0.8083693790435791, 'epoch': 28.571428571428573})

In [17]:
trainer.evaluate()

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.3193508386611938,
 'eval_accuracy': 0.3181818181818182,
 'eval_f1': 0.1536050156739812,
 'eval_runtime': 0.3478,
 'eval_samples_per_second': 63.256,
 'eval_steps_per_second': 8.626,
 'epoch': 28.571428571428573}

In [18]:
trainer.push_to_hub(new_model_name)

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/4.74M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/loha_fine_tuned_cb/commit/51bf7f3605761134574a4bb04dc3a2b179d21d85', commit_message='loha_fine_tuned_cb', commit_description='', oid='51bf7f3605761134574a4bb04dc3a2b179d21d85', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
# Example
hypothesis ="Valence je pomagal"
premise = "Valence praznoglavi, Valence krepostni kreten. Zakaj si ga tip raje ni zataknil v ustrezen del lastne titanske anatomije? Je morda mislil, da mi pomaga?"

In [20]:
# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
set_seed(42)
import torch

adapter_name = "lenatr99/" + new_model_name

tokenizer = AutoTokenizer.from_pretrained(adapter_name)
inputs = tokenizer(f"{premise} {hypothesis}", return_tensors="pt")
label = torch.tensor([1]).unsqueeze(0)  # Batch size 1

In [21]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
outputs = model(**inputs, labels=label)
logits = outputs.logits

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# Print prediction
logits.argmax().item()

1